In [48]:
from langchain_community.document_loaders import PyPDFDirectoryLoader

loader = PyPDFDirectoryLoader("./data_2")
docs = loader.load()
docs

[Document(page_content='Course Title\nPython for Data Science and Machine Learning Bootcamp\nThe Data Science Course: Complete Data Science Bootcamp 2024 \nR Programming A-Z™: R For Data Science With Real Exercises! \nStatistics for Data Science and Business Analysis \nData Science A-Z: Hands-On Exercises & ChatGPT Prize [2024] \nMachine Learning, Data Science and Generative AI with Python \nPython A-Z™: Python For Data Science With Real Exercises!\nComplete A.I. & Machine Learning, Data Science Bootcamp \nData Science and Machine Learning Bootcamp with R \nIntroduction to Machine Learning for Data Science \nBuild Responsive Real-World Websites with HTML and CSS \nHTML and CSS for Beginners - Build a Website & Launch ONLINE \nResponsive Web Design: HTML5 + CSS3 for Entrepreneurs 2018 \nWeb Design for Beginners: Real World Coding in HTML & CSS \nHTML5 and CSS3 Fundamentals \nLearn HTML5 Programming From Scratch \nBuild Websites from Scratch with HTML & CSS \nIntroduction to Cloud Comput

In [49]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1200, chunk_overlap = 250)
documents = text_splitter.split_documents(docs)

In [50]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

In [51]:
db = FAISS.from_documents(documents, OllamaEmbeddings())

In [52]:
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser



In [53]:
from dotenv import load_dotenv
import os

load_dotenv()
groq_api_key=os.getenv('GROQ_API_KEY')


In [54]:
llm = ChatGroq(groq_api_key = groq_api_key,
               model_name="Llama3-8b-8192")

In [72]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate

prompt=PromptTemplate.from_template(
"""
Act as helpfull Talent Transformation assistant and suggest the best Courses to take based on the given 
context, given {skills} and given {experience}. I want all the output in JSON format.


Prepare 3 sets of courses by total learning hours where Course type is Mandatory:
- Set 1 (1-day plan): Course duration < 3 hours
- Set 2 (3-day plan): Course duration between 3 and 20 hours
- Set 3 (5-day plan): Course duration > 20 hours

Also suggest 2 courses from Optional course type which suits the {experience} and {skills}

For each suggested course, provide the following:
- Course Name
- Course Link
- Course Length
- Learner Level


Please ensure consistency in providing course details and consider the inclusion of mandatory courses in the recommendations.

Suggest the courses only based on the context knowledge
Keep your course recomandation ground in the facts of the context.
If the context does not contain the suitable courses according to the given {skills} then return "No course Found"

Be as concise as possible when giving answers. Do not anounce that you will answering. 
Do not say "Let's get started", or "Based on the skills you provided, I would recommend the 
following courses to help you enhance your knowledge and skills : " or "Here are some courses"
and do not introduce your answer. Just write the your answer.

<context>
{context}
<context>

Skills: {skills}

Experience: {experience}

"""
)

In [56]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [73]:
retriever = db.as_retriever()

parser = JsonOutputParser()
setup = RunnableParallel(context=retriever, skills = RunnablePassthrough(), experience = RunnablePassthrough())

chain = setup | prompt | llm

In [74]:
response = chain.invoke({"skills": "python and AI",
              "experience": 2.5})

print(response.content)

[
  {
    "Course Name": "Git and GitHub for Beginners: Integration with Popular IDEs",
    "Course Link": "https://www.udemy.com/course/git-github-for-beginners-integration-with-popular-ides/?couponCode=LETSLEARNNOWPP",
    "Course Length": 2 hours,
    "Learner Level": Beginner
  },
  {
    "Course Name": "Git Advanced Commands",
    "Course Link": "https://www.udemy.com/course/git-advanced-commands/?couponCode=LETSLEARNNOWPP",
    "Course Length": 1 hour,
    "Learner Level": Intermediate
  },
  {
    "Course Name": "Git and GitHub: The Complete Git and GitHub Course",
    "Course Link": "https://www.udemy.com/course/git-github-the-complete-git-github-course/?couponCode=LETSLEARNNOWPP",
    "Course Length": 2 hours,
    "Learner Level": Beginner
  }
]

[
  {
    "Course Name": "Learn DevOps CI/CD with Jenkins using Pipelines and Docker",
    "Course Link": "https://www.udemy.com/course/learn-devops-ci-cd-with-jenkins-using-pipelines-and-docker/",
    "Course Length": 12 hours,
    "

In [77]:
response_2 = chain.invoke({"skills": "DevOps",
              "experience": 2})

print(response_2.content)


[
  {
    "set": "Set 1 (1-day plan)",
    "courses": [
      {
        "name": "Practical Approach to Version Control Git and GitHub",
        "link": "https://www.udemy.com/course/git-github-for-beginners-integration-with-popular-ides/?couponCode=LETSLEARNNOWPP",
        "length": 1,
        "level": "Beginner"
      }
    ]
  },
  {
    "set": "Set 2 (3-day plan)",
    "courses": [
      {
        "name": "Git & GitHub - The complete step-by-step guide from scratch",
        "link": "https://www.udemy.com/course/git-github-the-complete-git-github-course/?couponCode=LETSLEARNNOWPP",
        "length": 12,
        "level": "Beginner"
      },
      {
        "name": "Linux For Beginners covers Linux basics",
        "link": "https://www.udemy.com/course/linux-for-beginners-2021/?couponCode=LETSLEARNNOWPP",
        "length": 12,
        "level": "Beginner"
      }
    ]
  },
  {
    "set": "Set 3 (5-day plan)",
    "courses": [
      {
        "name": "Practical Approach to Version Cont

In [88]:
prompt_1 = PromptTemplate.from_template(
"""
**Prompt:**

"Act as helpfull Talent Transformation assistant and suggest the best Courses to take based on the given 
context and given Input.


Prepare 3 sets of courses by total learning hours:
- Set 1 (1-day plan): Course duration < 3 hours
- Set 2 (3-day plan): Course duration between 3 and 20 hours
- Set 3 (5-day plan): Course duration > 20 hours

**Input:**

* `user_experience`: A list of integers or a range of years representing the user's experience in their current role or a related field.
* `user_skills`: A list of strings or categories representing the user's skills and expertise.

**Dataset:**

* `dataset`: A dataframe containing information about various courses, with columns:
	+ `course_name`: A string representing the name of the course.
	+ `course_link`: A string representing the link to the course.
	+ `course_length`: An integer representing the duration of the course in hours.
	+ `course_type`: A string indicating whether the course is mandatory or optional.

**Output:**

For the given `user_experience` and `user_skills`, output a JSON object containing three sets of recommended courses for mandatory courses 
and one set for optional courses. The JSON object should have the following structure:
```json
{
  "recommended_courses": [
    {
      "course_name": "Course Title 1",
      "course_link": "https://example.com/course1",
      "course_length": 8,
      "course_type": "mandatory"
    },
    ...
  ],
  "optional_course": [
    {
      "course_name": "Course Title 2",
      "course_link": "https://example.com/course2",
      "course_length": 4,
      "course_type": "optional"
    }
  ]
}
```
"""
)

In [89]:
chain_1 = setup | prompt_1 | llm

In [90]:
response = chain_1.invoke({"user_skills": "Python and AI",
                           "user_experience": 2.5})

print(response.content)

KeyError: 'Input to PromptTemplate is missing variables {\'\\n  "recommended_courses"\'}.  Expected: [\'\\n  "recommended_courses"\'] Received: [\'context\', \'skills\', \'experience\']'